# day three
## part one
binary diagnostic decoding

In [1]:
package require Tcl 8.5

8.6.11

In [2]:
package require struct::list 1.8.4

1.8.4

In [3]:
set test_statuses [struct::list filter [split "00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010" \n] {string length} ]

00100 11110 10110 10111 10101 01111 00111 11100 10000 11001 00010 01010

In [4]:
set fl [open 03input]; set data [read $fl]; close $fl
set statuses [struct::list filter [split $data \n] {string length}]; unset data 
puts "statuses data in a list"

statuses data in a list


In [5]:
split [lindex $statuses 0] ""

1 1 1 1 0 0 0 0 0 1 0 1

In [6]:
proc make_report {report item} {
    set res {}
    foreach r $report i [split $item ""] {
        lappend res [incr r [ expr { ($i*2) -1 } ]]
    }
    return $res
}

In [7]:
set empty_report [struct::list repeat [string length [lindex $statuses 0]] 0]
set report [struct::list fold $statuses $empty_report make_report]

22 -4 8 26 46 -26 4 22 34 26 -36 -4

In [8]:
set gamma_report [string cat 0b [join [::struct::list mapfor d $report {expr { $d > 0 }}] "" ]]
set epsilon_report [string cat 0b [join [::struct::list mapfor d $report { expr { $d < 0 }}] "" ]] 

0b010001000011

In [9]:
puts "gamma [expr { $gamma_report }] epsilon [ expr { $epsilon_report }]"
expr { $gamma_report * $epsilon_report }

gamma 3004 epsilon 1091


3277364

## part two
oxigen rating and co2 rating are found in the list with a iterative process of elimination

In [10]:
package require lambda 1

1

In [11]:
proc is_1 { i item } {
    string index $item $i
}

proc is_0 { i item } {
    expr {![is_1 $i $item]}
}

proc count_bit_column { i } {
    lambda { i report item } {
        lassign $report zero_cnt one_cnt
        list [incr zero_cnt [is_0 $i $item]] [incr one_cnt [is_1 $i $item]]
    } $i
}

proc is_oxi { i report } {
    lassign $report 0_rep 1_rep

    if { $1_rep == 0 || $0_rep == 0} { return lambda {item} {expr 1} }
    
    if { $1_rep >= $0_rep } {
        list is_1 $i
    } else {
        list is_0 $i
    }
}

proc is_co2 { i report } {
    lassign $report 0_rep 1_rep
    if { $1_rep == 0 || $0_rep == 0} { return lambda {item} {expr 1} }
    
    if { $1_rep >= $0_rep } {
        list is_0 $i
    } else {
        list is_1 $i
    }
}


In [12]:
set oxi_statuses $statuses
for {set i 0; set max [string length [lindex $oxi_statuses 0]]} { $i < $max } {incr i } {
    if { [llength $oxi_statuses] == 1 } { break }
    set oxi_acc [struct::list fold $oxi_statuses [list 0 0] [count_bit_column $i]]
    set oxi_statuses [struct::list filter $oxi_statuses [is_oxi $i $oxi_acc]]
}
set oxi_rating [string cat 0b $oxi_statuses]
puts "oxi rating: $oxi_rating [expr {$oxi_rating}]"

oxi rating: 0b110111111111 3583


In [13]:
set co2_statuses $statuses
for {set i 0; set max [string length [lindex $co2_statuses 0]]} { $i < $max } {incr i } {
    if { [llength $co2_statuses] == 1 } { break }
    set co2_acc [struct::list fold $co2_statuses [list 0 0] [count_bit_column $i]]
    set co2_statuses [struct::list filter $co2_statuses [is_co2 $i $co2_acc]]
}
set co2_rating [string cat 0b $co2_statuses]
puts "co2 rating: $co2_rating [expr {$co2_rating}]"

co2 rating: 0b011001000001 1601


In [14]:
puts "life support rating: [expr {$oxi_rating * $co2_rating }]"

life support rating: 5736383
